In [ ]:
from src.model_predict import predict_with_models
from src.preprocessing import load_data
from src.forecasting import process_portfolio, predict_future_results
from src.feature_engineering import create_features
from src.model_fit import train_and_save_models
import os
os.makedirs('models/round8', exist_ok=True)
import warnings
warnings.filterwarnings("ignore")

In [7]:
data_dict = load_data('data/round_8/*.xlsx')

In [8]:
data_dict

{'Hong Kong International Airport':           Date  Total
 0   2023-01-08  30275
 1   2023-01-09  24796
 2   2023-01-10  24012
 3   2023-01-11  23273
 4   2023-01-12  23053
 ..         ...    ...
 824 2025-04-11  64252
 825 2025-04-12  62073
 826 2025-04-13  68078
 827 2025-04-14  64296
 828 2025-04-15  59413
 
 [829 rows x 2 columns],
 'Hong Kong-Zhuhai-Macao Bridge':           Date  Total
 0   2023-01-08   5321
 1   2023-01-09   4937
 2   2023-01-10   5617
 3   2023-01-11   6191
 4   2023-01-12   6699
 ..         ...    ...
 824 2025-04-11  31023
 825 2025-04-12  34587
 826 2025-04-13  55886
 827 2025-04-14  36194
 828 2025-04-15  31684
 
 [829 rows x 2 columns],
 'Lo Wu':           Date   Total
 0   2023-02-06   30319
 1   2023-02-07   32954
 2   2023-02-08   35022
 3   2023-02-09   34764
 4   2023-02-10   39235
 ..         ...     ...
 795 2025-04-11   82676
 796 2025-04-12   81797
 797 2025-04-13  119651
 798 2025-04-14   87925
 799 2025-04-15   81868
 
 [800 rows x 2 columns],
 '

In [9]:
processed_data_dict = {}
df_dict = {}
for name, data in data_dict.items():
    df ,processed_data = create_features(data)
    df_dict[name] = df
    processed_data_dict[name] = processed_data

In [10]:
processed_data_dict['Lo Wu'].tail()

,Date,Total,airport_sum,airport_pc,airport_mobile,bay_sum,bay_pc,bay_mobile,bridge_sum,bridge_pc,...,ex_rate_volatility_7d,rolling_7_mean,rolling_7_std,rolling_30_mean,ma_ratio_7_30,rolling_7_ex_rate_mean,month_sin,days_since_start,days_squared,weekly_growth
795,2025-04-11,82676,272,91,181,254,87,167,1650,341,...,0.001536,102405.428571,24045.320285,92956.000000,1.101655,0.934814,0.866025,795,632025,1.565046
796,2025-04-12,81797,257,76,181,234,74,160,1516,279,...,0.001561,95880.428571,21935.940318,93117.033333,1.029677,0.935214,0.866025,796,633616,1.144196
797,2025-04-13,119651,256,76,180,233,68,165,1410,234,...,0.002241,91276.285714,20847.890155,93166.200000,0.979715,0.936371,0.866025,797,635209,1.062043
798,2025-04-14,87925,280,116,164,306,127,179,1546,391,...,0.003553,88708.142857,14249.459831,94429.033333,0.939416,0.937814,0.866025,798,636804,1.119626
799,2025-04-15,81868,280,97,183,255,110,145,1501,377,...,0.004477,88046.714286,14148.234409,94071.933333,0.935951,0.939686,0.866025,799,638401,1.127825


In [11]:
df_dict['Lo Wu'].tail()

,Date,Total,ex,airport_sum,airport_pc,airport_mobile,bay_sum,bay_pc,bay_mobile,bridge_sum,...,ex_rate_volatility_7d,rolling_7_mean,rolling_7_std,rolling_30_mean,ma_ratio_7_30,rolling_7_ex_rate_mean,month_sin,days_since_start,days_squared,weekly_growth
795,2025-04-11,82676,0.9429,272,91,181,254,87,167,1650,...,0.001536,102405.428571,24045.320285,92956.000000,1.101655,0.934814,0.866025,795,632025,1.565046
796,2025-04-12,81797,0.9429,257,76,181,234,74,160,1516,...,0.001561,95880.428571,21935.940318,93117.033333,1.029677,0.935214,0.866025,796,633616,1.144196
797,2025-04-13,119651,0.9429,256,76,180,233,68,165,1410,...,0.002241,91276.285714,20847.890155,93166.200000,0.979715,0.936371,0.866025,797,635209,1.062043
798,2025-04-14,87925,0.9429,280,116,164,306,127,179,1546,...,0.003553,88708.142857,14249.459831,94429.033333,0.939416,0.937814,0.866025,798,636804,1.119626
799,2025-04-15,81868,0.9430,280,97,183,255,110,145,1501,...,0.004477,88046.714286,14148.234409,94071.933333,0.935951,0.939686,0.866025,799,638401,1.127825


In [12]:
cumulative_dict = {
    'Hong Kong International Airport': 0.8,
    'Hong Kong-Zhuhai-Macao Bridge': 0.85,
    'Lo Wu': 0.8,
    'Lok Ma Chau Spur Line': 0.8,
    'Shenzhen Bay': 0.85
}

In [15]:
models = train_and_save_models(processed_data_dict, cumulative_thresholds=cumulative_dict, min_features=10, model_dir='models/round8')

Hong Kong International Airport 最佳模型已保存：models/round8/Hong Kong International Airport_best_catboost.pkl (MAPE=0.0458)
特征选择：['lag_1', 'is_hk_holiday', 'lag_2', 'rolling_7_mean', 'lag_7', 'days_since_start', 'weekday', 'days_squared', 'airport_pc', 'ma_ratio_7_30', 'bridge_pc', 'post_cn_holiday', 'lag_4', 'lag_21', 'lowu_mobile', 'lag_14', 'bay_mobile', 'month_sin', 'lag_5', 'bay_sum', 'lowu_pc', 'day', 'rolling_30_mean', 'hk_map_pc_lag_7', 'year', 'hk_weather_mobile_lag_7', 'rolling_7_std']
Hong Kong-Zhuhai-Macao Bridge 最佳模型已保存：models/round8/Hong Kong-Zhuhai-Macao Bridge_best_stacking.pkl (MAPE=0.1128)
特征选择：['lag_1', 'day', 'bridge_pc', 'bridge_mobile', 'rolling_7_std', 'ex_rate_change_14d', 'ex_rate_lag_14', 'bridge_sum', 'hk_hotel_sum_lag_5', 'lag_21', 'bay_sum', 'pc_mobile_lag_7', 'bay_mobile', 'ma_ratio_7_30', 'lag_7', 'ex_rate_volatility_7d', 'rolling_7_mean', 'hk_pc_lag_7', 'rain', 'lag_2', 'bay_pc', 'hk_show_sum_lag_5', 'total_change_7d', 'hk_food_mobile_lag_5', 'baidu_pc_lag_7',

In [19]:
os.makedirs('models/round8_adjusted', exist_ok=True)
new_cumulative_dict = {
    'Hong Kong International Airport': 0.9,
    'Hong Kong-Zhuhai-Macao Bridge': 0.8,
    'Lo Wu': 0.9,
    'Lok Ma Chau Spur Line': 0.85,
    'Shenzhen Bay': 0.8
}

In [20]:
models_adjusted = train_and_save_models(processed_data_dict, cumulative_thresholds=new_cumulative_dict, min_features=10,
                               model_dir='models/round8_adjusted')

Hong Kong International Airport 最佳模型已保存：models/round8_adjusted/Hong Kong International Airport_best_catboost.pkl (MAPE=0.0466)
特征选择：['lag_1', 'is_hk_holiday', 'lag_2', 'rolling_7_mean', 'lag_7', 'days_since_start', 'weekday', 'days_squared', 'airport_pc', 'ma_ratio_7_30', 'bridge_pc', 'post_cn_holiday', 'lag_4', 'lag_21', 'lowu_mobile', 'lag_14', 'bay_mobile', 'month_sin', 'lag_5', 'bay_sum', 'lowu_pc', 'day', 'rolling_30_mean', 'hk_map_pc_lag_7', 'year', 'hk_weather_mobile_lag_7', 'rolling_7_std', 'lag_3', 'ex_rate_lag_5', 'airport_mobile', 'hk_weather_pc_lag_14', 'hk_hotel_sum_lag_7', 'bay_pc', 'hk_weather_sum_lag_5', 'ex_rate_lag_7', 'hk_pc_lag_7', 'hk_map_sum_lag_5', 'baidu_pc_lag_7', 'month', 'baidu_sum_lag_14', 'bridge_sum', 'lag_30', 'hk_show_sum_lag_7', 'hk_map_mobile_lag_7', 'lowu_sum', 'hk_show_pc_lag_5', 'hk_food_sum_lag_7', 'total_change_7d', 'hk_show_sum_lag_5', 'hk_food_pc_lag_14', 'hk_shopping_pc_lag_14', 'hk_weather_sum_lag_7', 'hk_pc_lag_5']
Hong Kong-Zhuhai-Macao Brid

In [17]:
step_list = [1, 1, 1, 3, 1]
result_dict = {}  # 用于存储所有预测结果
for (name, data), step in zip(data_dict.items(), step_list):
    if name == 'Hong Kong International Airport' or name == 'Lo Wu':
        result = process_portfolio(name, data, step)
        result_dict[name] = result
        print(f"{name} 模型训练完成")
        print(f'MAPE: {result["metrics"]}')
    else:
        pass

Hong Kong International Airport 模型训练完成
MAPE:        ETS   SARIMA  AutoTBATS  DynamicOptimizedTheta      MSTL  Ensemble
1  0.04093  0.04798   0.044758               0.043724  0.040201  0.064715
Lo Wu 模型训练完成
MAPE:         ETS   SARIMA  AutoTBATS  DynamicOptimizedTheta      MSTL  Ensemble
1  0.081122  0.07127   0.068634               0.075555  0.076323  0.093972


In [16]:
predictions = predict_with_models(df_dict, models, 5)

Hong Kong International Airport 预测结果: Date
2025-04-16    57485.613972
2025-04-17    59353.468371
2025-04-18    68654.300084
2025-04-19    72383.867942
2025-04-20    74298.142845
Name: Total, dtype: float64 (MAPE=0.0458)
Hong Kong-Zhuhai-Macao Bridge 预测结果: Date
2025-04-16    35802.894919
2025-04-17    48951.825699
2025-04-18    57853.966886
2025-04-19    55969.232976
2025-04-20    62781.519320
Name: Total, dtype: float64 (MAPE=0.1128)
Lo Wu 预测结果: Date
2025-04-16     80005.864494
2025-04-17     81161.492865
2025-04-18    115283.889280
2025-04-19    133267.441258
2025-04-20    135300.007658
Name: Total, dtype: float64 (MAPE=0.0654)
Lok Ma Chau Spur Line 预测结果: Date
2025-04-16     62995.796781
2025-04-17     61767.326648
2025-04-18     93043.650619
2025-04-19    100598.471773
2025-04-20    104200.174683
Name: Total, dtype: float64 (MAPE=0.0742)
Shenzhen Bay 预测结果: Date
2025-04-16    48479.811287
2025-04-17    50284.700857
2025-04-18    74047.403307
2025-04-19    82346.812197
2025-04-20    86

{'Hong Kong International Airport': (Date
  2025-04-16    57485.613972
  2025-04-17    59353.468371
  2025-04-18    68654.300084
  2025-04-19    72383.867942
  2025-04-20    74298.142845
  Name: Total, dtype: float64,
  0.04576704586667869),
 'Hong Kong-Zhuhai-Macao Bridge': (Date
  2025-04-16    35802.894919
  2025-04-17    48951.825699
  2025-04-18    57853.966886
  2025-04-19    55969.232976
  2025-04-20    62781.519320
  Name: Total, dtype: float64,
  0.11282904815048005),
 'Lo Wu': (Date
  2025-04-16     80005.864494
  2025-04-17     81161.492865
  2025-04-18    115283.889280
  2025-04-19    133267.441258
  2025-04-20    135300.007658
  Name: Total, dtype: float64,
  0.06539578588225783),
 'Lok Ma Chau Spur Line': (Date
  2025-04-16     62995.796781
  2025-04-17     61767.326648
  2025-04-18     93043.650619
  2025-04-19    100598.471773
  2025-04-20    104200.174683
  Name: Total, dtype: float64,
  0.07423221662682518),
 'Shenzhen Bay': (Date
  2025-04-16    48479.811287
  2025-0

In [21]:
predictions_adjusted = predict_with_models(df_dict, models_adjusted, 5)

Hong Kong International Airport 预测结果: Date
2025-04-16    58803.584932
2025-04-17    60907.926516
2025-04-18    67175.949094
2025-04-19    72190.289474
2025-04-20    76624.090639
Name: Total, dtype: float64 (MAPE=0.0466)
Hong Kong-Zhuhai-Macao Bridge 预测结果: Date
2025-04-16    29182.814482
2025-04-17    29259.190207
2025-04-18    47719.240538
2025-04-19    78596.736998
2025-04-20    82085.837187
Name: Total, dtype: float64 (MAPE=0.0759)
Lo Wu 预测结果: Date
2025-04-16     80789.724581
2025-04-17     81416.633474
2025-04-18    118256.854192
2025-04-19    134314.160055
2025-04-20    135279.220396
Name: Total, dtype: float64 (MAPE=0.0667)
Lok Ma Chau Spur Line 预测结果: Date
2025-04-16     77390.169709
2025-04-17     78069.478213
2025-04-18    104344.353053
2025-04-19    101728.478950
2025-04-20    104562.548643
Name: Total, dtype: float64 (MAPE=0.0811)
Shenzhen Bay 预测结果: Date
2025-04-16    43139.248773
2025-04-17    44837.430381
2025-04-18    67454.773349
2025-04-19    80263.281818
2025-04-20    86

In [18]:
results = predict_future_results(result_dict, 5)

Hong Kong International Airport: 未来5天预测结果 [57112.33, 59536.11332400061, 65671.38968971967, 63138.58005906127, 72254.79391545194]
MAPE: 0.040201438710712134 MODEL: MSTL
Lo Wu: 未来5天预测结果 [81055.33, 80514.43875032206, 82319.69424111613, 98729.4990999629, 125720.39729896402]
MAPE: 0.06863417646390357 MODEL: AutoTBATS


{'Hong Kong International Airport': ([57112.33,
   59536.11332400061,
   65671.38968971967,
   63138.58005906127,
   72254.79391545194],
  0.040201438710712134,
  'MSTL'),
 'Lo Wu': ([81055.33,
   80514.43875032206,
   82319.69424111613,
   98729.4990999629,
   125720.39729896402],
  0.06863417646390357,
  'AutoTBATS')}